# Skill 3: Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

Now that we know (from the prior Notebook) how to query tabular data on a CSV file and how to perform data analysis with Python, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with any SQL database like Synapse for example.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from common.prompts import MSSQL_AGENT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
#os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

# Install MS SQL DB driver in your machine

Use `lsb_release -a` to verify OS version details

In [3]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.6 LTS
Release:	20.04
Codename:	focal


## Using AML Instance


You might need the driver installed in order to talk to the SQL DB, so run the below cell once. Then restart the kernel and continue<br>
[Microsoft Learn Reference](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline)

In [4]:
!sudo ./download_odbc_driver.sh

Downloading...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  32766      0 --:--:-- --:--:-- --:--:-- 32766
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0   2617      0 --:--:-- --:--:-- --:--:--  2617
Updating apt-get..
Get:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Ign:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Get:2 file:/var/nccl-repo-2.2.13-ga-cuda9.2  Release [574 B]                   
Hit:3 http://azure.archive.ubuntu.com/ubuntu focal InRelease                   
Hit:4 http://azure.archive.ubuntu.com/ubuntu focal-updates InRelease           
Hit:5 http://azure.archive.ubuntu.com/ubuntu focal-backports InRelease         
Hit:6 http://azure.archive.ubuntu.com/ubuntu focal-security

## Using Dev Container

You might need the driver installed in order to talk to the SQL DB, so run the below cell once. Then restart the kernel and continue<br>
[Microsoft Learn Reference](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Cdebian17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline#17)

In [5]:
!chmod +x ./download_odbc_driver_dev_container.sh
!./download_odbc_driver_dev_container.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  35219      0 --:--:-- --:--:-- --:--:-- 36407
-----BEGIN PGP PUBLIC KEY BLOCK-----
Version: GnuPG v1.4.7 (GNU/Linux)

mQENBFYxWIwBCADAKoZhZlJxGNGWzqV+1OG1xiQeoowKhssGAKvd+buXCGISZJwT
LXZqIcIiLP7pqdcZWtE9bSc7yBY2MalDp9Liu0KekywQ6VVX1T72NPf5Ev6x6DLV
7aVWsCzUAF+eb7DC9fPuFLEdxmOEYoPjzrQ7cCnSV4JQxAqhU4T6OjbvRazGl3ag
OeizPXmRljMtUUttHQZnRhtlzkmwIrUivbfFPD+fEoHJ1+uIdfOzZX8/oKHKLe2j
H632kvsNzJFlROVvGLYAk2WRcLu+RjjggixhwiB+Mu/A8Tf4V6b+YppS44q8EvVr
M+QvY7LNSOffSO6Slsy9oisGTdfE39nC7pVRABEBAAG0N01pY3Jvc29mdCAoUmVs
ZWFzZSBzaWduaW5nKSA8Z3Bnc2VjdXJpdHlAbWljcm9zb2Z0LmNvbT6JATUEEwEC
AB8FAlYxWIwCGwMGCwkIBwMCBBUCCAMDFgIBAh4BAheAAAoJEOs+lK2+EinPGpsH
/32vKy29Hg51H9dfFJMx0/a/F+5vKeCeVqimvyTM04C+XENNuSbYZ3eRPHGHFLqe
MNGxsfb7C7ZxEeW7J/vSzRgHxm7ZvESisUYRFq2sgkJ+HFERNrqfci45bdhmrUsy
7SWw9ybxdFOkuQoyKD3tBmiGfONQMlBaOMWdAsic9

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [6]:
#Keyvault
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace

ml_client = MLClient.from_config(credential=DefaultAzureCredential(), path="workspace.json")
ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

import uuid

keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="username", value = os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"])


Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fec318f9c30>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [7]:
username=keyvault.get_secret(name="username")

In [8]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import os

# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': username,
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)


InterfaceError: (pyodbc.InterfaceError) ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'admin_yk'. Reason: Azure Active Directory only authentication is enabled. Please contact your system administrator. (18456) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [ ]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    #Createse the table in Azure SQL
    with engine.connect() as conn:
        # Execute the create table SQL statement
        conn.execute(text(create_table_sql))
        print("Table", table_name, "successfully created")
    # Insert data into SQL Database
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

# Query with LLM

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right values for the ENV variables and it will connect succesfully.

In [ ]:
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000)

In [ ]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

In [ ]:
# Natural Language question (query)
QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
"""

### SQL Agent

LangChain has a SQL Agent which provides a more flexible way of interacting with SQL Databases than a chain. The main advantages of using the SQL Agent are:

    It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
    It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
    It can query the database as many times as needed to answer the user question.
    It will save tokens by only retrieving the schema from relevant tables.

To initialize the agent we’ll use the `create_sql_agent` constructor. This agent uses the SQLDatabaseToolkit which contains tools to:

    Create and execute queries
    Check query syntax
    Retrieve table descriptions
    … and more

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    agent_type="openai-tools",
    verbose=True,

)

In [ ]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.tools

In [ ]:
try:
    response = agent_executor.invoke(QUESTION) 
except Exception as e:
    response = str(e)

In [ ]:
printmd(response["output"])

**IMPORTANT NOTE**: If you don't specify the column name on the question, runing the above cell multiple times will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT models.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. LLMs have been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to create a custom agent that connects to the internet using BING SEARCH API to answer questions grounded on search results with citations. Basically a clone of Bing Chat.